In [1]:
from google.colab import drive
from nltk.corpus import stopwords
import nltk
from collections import defaultdict
import re
from nltk.stem.porter import PorterStemmer
import heapq
import json
import gc
porter = PorterStemmer()

In [2]:
nltk.download('stopwords')
stop_words = stopwords.words('english')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [3]:
def remove_symbols(line):
    return re.sub('[^A-Za-z0-9\s]+', '', line).lower()

"""
we are using this to change list into set while dumping json into file
"""
class SetEncoder(json.JSONEncoder):
  def default(self, obj):
    if isinstance(obj, set):
      return list(obj)
    return json.JSONEncoder.default(self, obj)

In [7]:
drive.mount('/content/drive')

Mounted at /content/drive


In [8]:
import csv
import sys
csv.field_size_limit(sys.maxsize) # if we don't do this, we won't be able to read whole line ( try to comment this line for action )

9223372036854775807

In [17]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [18]:
# going to our folder
% cd /content/drive/MyDrive/
# % rm -rf OP*

/content/drive/MyDrive


In [19]:
 % ls # just to confirm and see if we have file
#  gutenberg_data.csv => has 15331 LINES
# BLOCK_SIZE = 10000000
BLOCK_SIZE = 100000 # 1 time around 18 secs, TOTAL TIME TAKEN => 4h25m20s

'1Scholarship decleration form (2).gdoc'
 201952223_10.pdf
'201952223_1 (1).pdf'
 201952223_11.pdf
'201952223_1 (2).pdf'
 201952223_12.pdf
 201952223_13.pdf
 201952223_14.pdf
 201952223_15.pdf
'201952223 (1).pdf'
 201952223_1.pdf
'201952223_2 (1).pdf'
'201952223_2 (2).pdf'
 201952223_2.pdf
'201952223_3 (1).pdf'
'201952223_3 (2).pdf'
 201952223_3.pdf
'201952223_4 (1).pdf'
 201952223_4.pdf
'201952223_5 (1).pdf'
 201952223_5.pdf
 201952223_6.pdf
 201952223_7.pdf
 201952223_8.pdf
 201952223_9.pdf
'201952223_Assignment 3 and 4.pdf'
 201952223_CS206_MidSem.pdf
'201952223__CS262 Lab Assignment 4.pdf'
'201952223 CS263 Lab 3.pdf'
'201952223 CS263 Lab 4.pdf'
'201952223 CS263 Lab 5.pdf'
'201952223 CS263 Lab 6.pdf'
'201952223 CS263 Lab_Assignment10.pdf'
'201952223 CS263 Lab_Assignment11.pdf'
'201952223 CS263 Lab_Assignment7.pdf'
'201952223 CS263 Lab_Assignment8.pdf'
'201952223 CS263 Lab_Assignment9.pdf'
'201952223_CS268_Assignment 5 and 6.pdf'
'201952223_CS268_Assignment 7 and 8.pdf'
'201952223_CS

In [29]:
def bsbi():
  freq_dict = defaultdict(set)
  with open('gutenberg_data.csv') as f:
    next(f) # just skipping first line(column headers)
    csv_file = csv.reader(f)
    total_files = 0
    i = 0
    current_block = 0
    for line in csv_file:
      title, author, link, id, bookshelf, text = line
      print(id," => " ,author)
      i += 1
      # for testing 
      if i == 100:
        break
      for word in text.split():
        word = remove_symbols(word)
        if word and word not in stop_words:
          word = porter.stem(word)
          if word not in freq_dict:
            # if word is not added before, we will increase block size too
            current_block += 1
          """
          note: __contains__ will just check if doc is already there.
          It's not exactly needed as we are using set, but we are checking it to manage block size
          also searching in sets is faster as they are hashed (while lists are not hashed)
          """
          if not freq_dict[word].__contains__(id):
            freq_dict[word].add(id)
            current_block += 1
        if current_block >= BLOCK_SIZE:
          # LETS DO THE WRITE OPERATION
          sorted_list = sorted(freq_dict.items(), key=lambda _: _[0]) # sorting by word_id
          with open(f'./IR2_output/output{total_files}.txt', 'w') as  f:
            # json.dump(freq_dict, f, cls=SetEncoder)
            for word_id, docs in sorted_list:
              f.write(word_id)
              for doc_id in docs:
                f.write(f' {doc_id}')
              f.write('\n')
          current_block = 0
          freq_dict.clear()
          total_files += 1
          print(i, ' rows done')
          if total_files == 4:
            return
      
    sorted_list = sorted(freq_dict.items(), key=lambda _: _[0]) # sorting by word_id
    # this is for last values
    # TODO:: DO IT BY FN SO NO REPEATATION OF CODE
    if len(sorted_list) > 0:
      with open(f'./IR2_output/output{total_files}.txt', 'w') as  f:
        # json.dump(freq_dict, f, cls=SetEncoder)
        for word_id, docs in sorted_list:
          f.write(word_id)
          for doc_id in docs:
            f.write(f' {doc_id}')
          f.write('\n')
      current_block = 0
      freq_dict.clear()
      total_files += 1
bsbi()

17748  =>  William T. Hornaday
34110  =>  A. R. Harding
38315  =>  Édouard Cuyer
30221  =>  Color Photography, Vol. 1, No. 1 Various
32947  =>  A. Mueller
34063  =>  E. N. Woodcock
31751  =>  Frank M. Chapman
34076  =>  
34984  =>  Lucas, Shinn, Smallwood, and Whitney
13249  =>  William T. Hornaday
34501  =>  A. R. Harding
37787  =>  W. H. Hudson
38687  =>  De Gubernatis
36922  =>  G. Murray Levick
34131  =>  William S. Furneaux
15  rows done
30000  =>  Chester A. Reed
37122  =>  Grinnell et al.
27887  =>  Ernest Thompson Seton
33044  =>  Max C. Thompson
13325  =>  Sir James Emerson Tennent
19550  =>  Robert Armitage Sterndale
37009  =>  Clarence Moores Weed
38013  =>  Frederic A. Lucas
23  rows done
34098  =>  Elmer Harry Kreps
33874  =>  Edward Saunders
215  =>  Jack London
7766  =>  Ouida
35450  =>  Harrison Weir
38777  =>  Albert Payson Terhune
30667  =>  Arthur Scott Bailey
24350  =>  Edith Wharton
26500  =>  Edward Jesse
32069  =>  Helen Hunt Jackson
32300  =>  Dinks, W. N. Hutch

In [30]:
file_names = [f'./IR2_output/output{i}.txt' for i in range(947)]
file_pointers = [open(i) for i in file_names]

In [32]:


# here we are using yeild so we are JUST READING ONE LINE at a time

def decorated_file(f, key):
  for line in f:
    yield (key(line), line)

files = map(open, file_names)
outfile = open('./IR2_output/merged_out.txt', 'w')

def key_fn(line):
    return line.split(' ', 2)[0] # returning word_id

"""
so if there'd be 2 files for example :
a 1 2 3
b 3 4 5

and 

a 6 7 8
b 8 9 10

o/p would be of the form :
a 1 2 3
a 6 7 8
b 3 4 5
b 8 9 10

TIME: 6m 14s (for ~992 files of size ~650kb each)
"""
prev = ''
# The simple sorting version would be like below
for line in heapq.merge(*[decorated_file(f, key_fn) for f in files]):
  
  if prev != line[0]:
    # if we have new word, make sure to add new line at first
    outfile.write(f'\n{line[1].strip()}')
    prev = line[0]
  # if we have same word yet, put a space and add other ids
  else:
    # line[1][len(line[0]):] => We are removing the word_id string and then writing the line
    outfile.write(f' {line[1][len(line[0]):].strip()}')
for i in file_pointers:
  i.close()
outfile.close()

In [34]:
for i in file_pointers:
  i.close()
outfile.close()